In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from tqdm import tqdm

In [ ]:
url = "https://www.opinautos.com.br"

def get_soup(url):
    page = urlopen(url)
    html_bytes = page.read()
    html = html_bytes.decode("utf-8")
    return BeautifulSoup(html, "html.parser")

def flatten_list(nested_list):
    return [item for sublist in nested_list for item in sublist]

In [ ]:
href = "/br/escolhermarca/"
soup = get_soup(url+href)

In [ ]:
marcas = soup.find(id="restoMarcas").find_all("a")

In [ ]:
todos_carros = []
for marca in marcas:
    href = marca['href']
    soup_marca = get_soup(url+href)
    carros = soup_marca.find('div', class_='ColumnsMax5').find_all('a')
    
    qtd_carros = len(carros)
    
    for carro in tqdm(carros, total=qtd_carros, desc=href.split('/')[1]):
        carro = get_soup(url + carro['href'] + "/info/manuais")
        manuais = carro.find('div', class_='manuals-box')
        if manuais: todos_carros.extend(manuais.find_all("a", recursive=False))

In [64]:
df = pd.DataFrame(list(map(lambda x: x['href'].split('/')[1:] + [url+x['href']], todos_carros)), columns=['marca', 'modelo', 'info', 'manual', 'ano', 'link'])
df = df.drop(columns=['info', 'manual'])

In [70]:
df['marca_count'] = df.groupby(['marca'])['modelo'].transform('count')

In [75]:
df[df['marca'] == 'hyundai']

,marca,modelo,ano,link,marca_count
468,hyundai,creta,2022,https://www.opinautos.com.br/hyundai/creta/inf...,3
469,hyundai,hb20,2022,https://www.opinautos.com.br/hyundai/hb20/info...,3
470,hyundai,veloster,2022,https://www.opinautos.com.br/hyundai/veloster/...,3


In [72]:
df.to_csv('manuais_carro.csv')